In [1]:
import os
import io
from time import time
from datetime import datetime
import itertools
os.environ["CUDA_VISIBLE_DEVICES"]="-1" #set to -1 to disable gpu (on fresh kernel)

In [2]:
import tensorflow as tf
import keras
import keras.backend as K

Using TensorFlow backend.


In [3]:
K.tensorflow_backend._get_available_gpus()
# tf.test.is_gpu_available()

[]

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
os.chdir(r"../Immediate Result")

In [6]:
df = pd.read_csv(r"cleandf_40days_930_to_1830_lin_interpolated.csv")

In [7]:
df[:10]

CAMERA_ID            TIMESTAMP   GEO_LON    GEO_LAT  DENSITY_VALUE
0          4  2010-10-11 09:30:00 -0.227278  51.491661   53547.356500
1          4  2010-10-11 10:00:00 -0.227278  51.491661   46199.700875
2          4  2010-10-11 10:30:00 -0.227278  51.491661   39602.705900
3          4  2010-10-11 11:00:00 -0.227278  51.491661   27179.430333
4          4  2010-10-11 11:30:00 -0.227278  51.491661   23527.222300
5          4  2010-10-11 12:00:00 -0.227278  51.491661   19940.461500
6          4  2010-10-11 12:30:00 -0.227278  51.491661   20405.220000
7          4  2010-10-11 13:00:00 -0.227278  51.491661   21775.247000
8          4  2010-10-11 13:30:00 -0.227278  51.491661   18727.309900
9          4  2010-10-11 14:00:00 -0.227278  51.491661   17784.156700

In [8]:
# df.DENSITY_VALUE = np.log(df.DENSITY_VALUE)
df.DENSITY_VALUE -= df.DENSITY_VALUE.min()
df.DENSITY_VALUE /= df.DENSITY_VALUE.max() # scaling to [0,1]
df = df.pivot(index="TIMESTAMP",columns="CAMERA_ID",values="DENSITY_VALUE")
L = int(0.75 * len(df)) #3/4 of the length of data (over time) to be used as train and what ever comes after this would be for evaluation

In [11]:
df

CAMERA_ID                 4         5         6         7         8    \
TIMESTAMP                                                               
2010-10-11 09:30:00  0.832721  0.736465  0.673375  0.875612  1.000000   
2010-10-11 10:00:00  0.717267  0.677046  0.539202  0.536337  0.928692   
2010-10-11 10:30:00  0.613608  0.580922  0.429255  0.479447  0.857383   
2010-10-11 11:00:00  0.418400  0.434257  0.266390  0.339875  0.725234   
2010-10-11 11:30:00  0.361012  0.404033  0.204576  0.306596  0.321575   
2010-10-11 12:00:00  0.304653  0.428367  0.183852  0.274417  0.289617   
2010-10-11 12:30:00  0.311956  0.304918  0.206830  0.266901  0.291421   
2010-10-11 13:00:00  0.333483  0.316085  0.242929  0.242035  0.285842   
2010-10-11 13:30:00  0.285591  0.366087  0.161221  0.244930  0.305829   
2010-10-11 14:00:00  0.270771  0.277595  0.120844  0.226747  0.304305   
2010-10-11 14:30:00  0.341074  0.261700  0.137740  0.218907  0.317512   
2010-10-11 15:00:00  0.340054  0.229100  0.138745  0.260715  0.348867   
2010-10-11 15:30:00  0.423740  0.220598  0.203397  0.377745  0.419855   
2010-10-11 16:00:00  0.446975  0.241334  0.160889  0.402249  0.513418   
2010-10-11 16:30:00  0.541662  0.373131  0.179553  0.452027  0.634326   
2010-10-11 17:00:00  0.623828  0.570275  0.421990  0.577903  0.686852   
2010-10-11 17:30:00  0.560180  0.649326  0.449701  0.601893  0.650849   
2010-10-11 18:00:00  0.479465  0.583546  0.507219  0.368292  0.548068   
2010-10-11 18:30:00  0.628765  0.642776  0.650817  0.734953  0.703240   
2010-10-12 09:30:00  0.406132  0.432930  0.527475  0.311408  0.440725   
2010-10-12 10:00:00  0.394718  0.422405  0.492325  0.321751  0.387691   
2010-10-12 10:30:00  0.392652  0.406920  0.370317  0.259585  0.292303   
2010-10-12 11:00:00  0.350642  0.374163  0.342122  0.231752  0.261941   
2010-10-12 11:30:00  0.377405  0.408138  0.316887  0.222012  0.282189   
2010-10-12 12:00:00  0.329586  0.327265  0.299036  0.263719  0.256575   
2010-10-12 12:30:00  0.283286  0.299494  0.354648  0.235572  0.246865   
2010-10-12 13:00:00  0.267242  0.295020  0.316561  0.288581  0.269757   
2010-10-12 13:30:00  0.223575  0.271862  0.322887  0.327436  0.297633   
2010-10-12 14:00:00  0.185948  0.245146  0.319398  0.343789  0.259786   
2010-10-12 14:30:00  0.181039  0.239531  0.266494  0.309499  0.328422   
...                       ...       ...       ...       ...       ...   
2010-11-21 13:30:00  0.113510  0.230744  0.109129  0.061064  0.079682   
2010-11-21 14:00:00  0.156925  0.237202  0.288286  0.073161  0.080484   
2010-11-21 14:30:00  0.067753  0.219193  0.094437  0.070939  0.168746   
2010-11-21 15:00:00  0.068327  0.203660  0.091436  0.062752  0.126965   
2010-11-21 15:30:00  0.092746  0.214257  0.125261  0.092806  0.169692   
2010-11-21 16:00:00  0.286689  0.291706  0.427269  0.530477  0.516520   
2010-11-21 16:30:00  0.466970  0.267430  0.619214  0.550645  0.730653   
2010-11-21 17:00:00  0.432327  0.219004  0.492477  0.384737  0.669440   
2010-11-21 17:30:00  0.354311  0.210346  0.400935  0.314949  0.639246   
2010-11-21 18:00:00  0.327625  0.225696  0.376124  0.277541  0.592714   
2010-11-21 18:30:00  0.277184  0.245703  0.337465  0.254163  0.667740   
2010-11-22 09:30:00  0.471847  0.382561  0.208767  0.346169  0.157123   
2010-11-22 10:00:00  0.240212  0.281684  0.139550  0.162721  0.128547   
2010-11-22 10:30:00  0.190738  0.288075  0.118425  0.229427  0.138285   
2010-11-22 11:00:00  0.156867  0.247803  0.116203  0.093070  0.151848   
2010-11-22 11:30:00  0.117920  0.236391  0.062228  0.097277  0.110819   
2010-11-22 12:00:00  0.101493  0.270458  0.130093  0.118414  0.322923   
2010-11-22 12:30:00  0.125826  0.234036  0.087783  0.139552  0.247877   
2010-11-22 13:00:00  0.078169  0.215753  0.104974  0.160689  0.211875   
2010-11-22 13:30:00  0.108529  0.268844  0.136086  0.181826  0.278216   
2010-11-22 14:00:00  0.132704  0.264259  0.148662  0.202963  0.272539   
2010-11-22 14:30:00  0.147792  0.268457  0.096696  0.29897

In [9]:
from keras import Sequential
from keras.layers import Dense, Activation, LSTM
from keras.optimizers import Adam, RMSprop
from keras import regularizers

from keras.layers import Input, Concatenate, Reshape, Lambda, Dropout
from keras.layers import TimeDistributed, Bidirectional
from keras.models import Model

from keras.callbacks import TensorBoard
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [10]:
TIMESTEPS = 19 * 7 #this should be played with to find a good one!
CAMERA_FEATURES = 58 #this is all the cameras we have. We focus on one at a time in this part of the study.
BATCH_SIZE = 1 #let's stick to STATEFUL RNN models with BATCH_SIZE of 1. We can try batch learning later but it kinda would be hard to implement the inference model (where it actually uses the model on incoming data to predict what happens next.)

In [33]:
#df must be pivoted df not the original one.
def get_data_generator(df, camera_id, TIMESTEPS, mode = "train"):
    if mode == "train": 
        while True:
            for i in range(0, L-TIMESTEPS): #first 3/4th of the time (30 days)
                X = np.reshape(df.loc[:,camera_id].iloc[i:i+TIMESTEPS].values, newshape=(BATCH_SIZE,TIMESTEPS,1) ) # 1 is the number of features we have which is one cause for each camera we have have itself.
                y = np.reshape(df.loc[:,camera_id].iloc[TIMESTEPS], newshape=(BATCH_SIZE,1)) #it is just one number but we reshape it as two dim tensor (1,1)
                yield (X,y) #in generator must return a tuple of input and output in training mode.
                
    elif mode == "eval": 
        while True:
            for i in range(L-TIMESTEPS, len(df) - TIMESTEPS): #remaining quarter of data (10 days)
                X = np.reshape(df.loc[:,camera_id].iloc[i:i+TIMESTEPS].values, newshape=(BATCH_SIZE,TIMESTEPS,1) ) # 1 is the number of features we have which is one cause for each camera we have have itself.
                y = np.reshape(df.loc[:,camera_id].iloc[TIMESTEPS], newshape=(BATCH_SIZE,1))
                yield (X,y) #in generator must return a tuple of input and output in training mode.
                
    elif mode == "predict": 
        while True:
            for i in range(L-TIMESTEPS, len(df) - TIMESTEPS): #same as eval without target y
                yield np.reshape(df.loc[:,camera_id].iloc[i:i+TIMESTEPS].values, newshape=(BATCH_SIZE,TIMESTEPS,1) )

In [18]:
#you can use my code here from the all cameras model I'm working on. 
#function returns layer objects you need to add to your model where you need an LSTM block.
#It supports stacking up to 3 lstms and bidirectional lstm wrapper.
#assuming the model = Sequential()
#layers = get_lstm_layers(params)
#for layer in layers:
#    model.add(layer)
def get_lstm_layers(params):
    """returns keras tensor output of a block of stateful lstms (potentially stacked)."""
    n = params["stack_lstm_count"]

    layers = []
    
    lstm1 = LSTM( params["lstm1_units"],
                     batch_input_shape=(1,params["timesteps"],params["dense_units"]), 
                     stateful=True,
                     return_sequences= True if n > 1 else False,
                     dropout=params["lstm1_dropout"],
                     recurrent_dropout=params["lstm1_recurrent_dropout"],
                     activity_regularizer=regularizers.l2(l=params["lstm1_l2reg"]),
                )
                 
    if "bidirectional_lstms" in params and params["bidirectional_lstms"] == True:
        lstm1 = Bidirectional(lstm1) #default mode is concat

    layers.append(lstm1)
                 
    if n == 1:
        return layers
        
    lstm2 = LSTM(params["lstm2_units"],
                     stateful=True,
                     return_sequences = True if n > 2 else False,
                     dropout=params["lstm2_dropout"],
                     recurrent_dropout=params["lstm2_recurrent_dropout"],
                     activity_regularizer=regularizers.l2(l=params["lstm2_l2reg"]),
                    )
                 
    if "bidirectional_lstms" in params and params["bidirectional_lstms"] == True:
        lstm2 = Bidirectional(lstm2) #default mode is concat  
    
    layers.append(lstm2)
                 
    if n == 2:
        return layers
                
    
    lstm3 = LSTM(params["lstm3_units"],
                stateful=True,
                return_sequences=False,
                dropout=params["lstm3_dropout"],
                recurrent_dropout=params["lstm3_recurrent_dropout"],
                activity_regularizer=regularizers.l2(l=params["lstm3_l2reg"]),
               )
                 
    if "bidirectional_lstms" in params and params["bidirectional_lstms"] == True:
        lstm3 = Bidirectional(lstm3) #default mode is concat  
                 
    layers.append(lstm3)
                 
    return layers

In [20]:
#I have been using Mean Exp Sq Error instead of Mean Sq Error (I am experimenting with both)
#Hoping exp would put more pressure on the model to capture spikes.

#You may need to change this 
#K is keras.backend

#Not sure if you need K.mean as it is only one value you predict not all cameras at once.
def exp_loss(yTrue,yPred):
    return K.mean(K.exp(K.square(yTrue - yPred,)))

In [21]:
#This is the latest parameters I have used. You need to change/modify things (at least account for camera_id)
paramss = []
paramss.append({"desc" : "testing_bidirectional", "timesteps" : 19*21, "loss" : exp_loss,
                "stack_lstm_count" : 2, "bidirectional_lstms" : True,
                "lstm1_units" : 80, "lstm1_dropout" : 0, "lstm1_recurrent_dropout" : 0, "lstm1_l2reg" : 0.,
                "lstm2_units" : 80, "lstm2_dropout" : 0.4, "lstm2_recurrent_dropout" : 0, "lstm2_l2reg" : 0.05,
                "dense_units" : 58*3, "dense_dropout" : 0.4, "dense_l2reg" : 0.001, "dense_activation" : "sigmoid"})

paramss.append({"desc" : "mimic_v1_stacked_after_refactor", "timesteps" : 19*21, "loss" : exp_loss,
                "stack_lstm_count" : 2, 
                "lstm1_units" : 80, "lstm1_dropout" : 0, "lstm1_recurrent_dropout" : 0, "lstm1_l2reg" : 0.,
                "lstm2_units" : 80, "lstm2_dropout" : 0.4, "lstm2_recurrent_dropout" : 0, "lstm2_l2reg" : 0.05,
                "dense_units" : 58*3, "dense_dropout" : 0.4, "dense_l2reg" : 0.001, "dense_activation" : "sigmoid"})

paramss.append({"desc" : "mimic_v1_stacked_with_mse_loss", "timesteps" : 19*21, "loss" : "mse",
                "stack_lstm_count" : 2, 
                "lstm1_units" : 80, "lstm1_dropout" : 0, "lstm1_recurrent_dropout" : 0, "lstm1_l2reg" : 0.,
                "lstm2_units" : 80, "lstm2_dropout" : 0.4, "lstm2_recurrent_dropout" : 0, "lstm2_l2reg" : 0.05,
                "dense_units" : 58*3, "dense_dropout" : 0.4, "dense_l2reg" : 0.001, "dense_activation" : "sigmoid"})

In [ ]:
def get_model(params):
    #TODO: implement you model here. and return the model object.
    # you should start with a simple model that uses a relatively small LSTM to predict.
    model = Sequential()
    
    
    return model

In [ ]:
CURRENT_TIME = datetime.now().strftime("%Y-%m-%d %H-%M")

for params in paramss: #this way you can manually/automatically define your hyperparams to run and then let it run all the configs
    #there are other ways for hyperparam tuning but I wrote it this way from scratch for my stuff and it seems to do its job.
    #we might need a final hyper param tuning were it does a grid search of some of the parameters or a smart state space search (hopefully implemented already in a tool like Microsoft NNI)
    
    model = get_model(params)

    model.compile(optimizer=Adam(lr=0.001,clipnorm=1.), loss=params["loss"], metrics=["mse"])

    print(model.summary())
    
    # If you are going to use tensorboard for visualization you need to pass a TensorBoard callback to the fit function.
    # The default one, plots train and evaluation losses separately.
    # I used this one:
    # https://stackoverflow.com/questions/47877475/keras-tensorboard-plot-train-and-validation-scalars-in-a-same-figure
    # so that it would plot them in the same figure.
    
    # LOGDIR = "logs/{}[{}]".format(params["desc"], CURRENT_TIME )     
    # tensorboard = TrainValTensorBoard(log_dir= LOGDIR)

    # One more thing about tensorboard callback: write_grads seems broken and not fixed yet. It would have been nice to see the gradients..
    # write_grads=True, histogram_freq= 2, batch_size=1 )

    #Earlystopping would stop training if the model validation (evaluation) loss does not improve over patience epochs for at least min_delta.
    es = EarlyStopping(monitor="val_loss", patience=10, min_delta=0.001, restore_best_weights=True)

    #logging parameters to text file
    with open('params_log_lstm_per_camera_for_london.txt', 'a') as f:
        print(params, file=f)
        
    train_gen = get_data_generator(df, params["camera_id"], params["timesteps"], mode = "train")
    #don't reuse generator as it might be messed up.
    
    val_gen = get_data_generator(df, params["camera_id"], params["timesteps"], mode="eval")
    
    history = model.fit_generator(train_gen,
                                  epochs = 250,
                                  steps_per_epoch= L-TIMESTEPS,
                                  validation_data = val_gen,
                                  validation_steps = len(df) - L, 
                                  callbacks=[es] #tensorboard goes here
                                 )
    
    del model 
    del history
    K.clear_session() #to hopefully prevent slow down after a few models have run..

### Notes